# 装饰器基础
首先，装饰器是可调用的对象，参数为被装饰的函数。装饰器用于给函数或对象动态的添加一些额外的功能，其最常见的实现方式是在装饰器内部定义一个函数实现对被装饰函数的功能增强并返回一个可调用对象来替代原函数或对象。

In [2]:
def deco(func):
    def inner():
        print("running inner()")
    return inner # 返回一个函数

@deco
def target():
    print('running target()')
    
print(target())
print(target) # target被替换为inner函数

running inner()
None
<function deco.<locals>.inner at 0x0000023BAAD7A510>


python中的@decorator的方式只是语法糖，上述写法其实也可以写为：target = deco(target)。  

虽然一般情况下，装饰器不会返回原函数，但是这种行为可以用于注册函数，如Web框架将URL处理函数添加到中央注册处。对于第6章的策略收集问题就可利用装饰器对策略进行收集而无需使用内省（需要强制将策略写在一个模块里，或是通过限制其名称利用globals()收集）。
## 装饰器运行时间
装饰器的另一特性是，在被装饰函数定义后立刻执行，也就是加载模块时。

# 闭包与对象作用域
由于一般情况下，函数装饰器在内部会重新定义一个‘增强’版的函数来返回替代原函数，因此这种嵌套一般说来都会用到闭包的特性。了解闭包前需要了解对象作用域规则。
## 对象作用域规则
对于函数体内赋值的变量，python将其视为局部变量。

In [3]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

#### 上述示例中，由于f2定义中有对b的赋值，python首先认为b是f2的局部变量，当运行时由于在b赋值前就需要读取b的值，因此会报错。
这种行为对应与js中需要var声明才将变量视为局部变量。但js在函数内可能忘记声明局部变量，可能误获取全局变量并对其赋值。python相当于强制如果想对全局变量进行赋值则必须使用global声明（否则被视为局部变量）。
# 闭包
闭包是指延伸了作用域的函数，其中包含函数定义体内引用的但没在函数体内定义的__非全局变量__，由此说来，类内定义的函数也算闭包，虽然不能直接调用，但是可以使用没在函数体内定义的非全局变量（实例变量或类变量）

In [1]:
class Averager():
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)
    
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

avg1 = Averager()
print(avg1(10))
print(avg1(11))

avg2 = make_averager()
print(avg2(10))
print(avg2(11))

10.0
10.5
10.0
10.5


## 自由变量
第一个使用类来维护历史数据，每次只需要调用Averager的可调用实例即可返回平均值。

第二个使用闭包，按理说嵌套的averager的函数被返回后，其外层的make_average的本地作用域也就消失了，__但是这里make_average的series变量仍然存在（非全局变量）且会作为自由变量被绑定到使用它的内层嵌套函数averager上__。

闭包其实是一种函数，它会保留定义在函数内存在的自由变量的绑定（延伸作用域的非全局变量），虽然自由变量的定义域消失了但仍可使用。

In [4]:
print(avg2.__code__.co_varnames) # 打印编译后的函数定义体的局部变量名属性
print(avg2.__code__.co_freevars) # 打印编译后的函数定义体的自由变量名属性
print(avg2.__closure__[0]) # __closure__属性保存cell对象组成的list 
print(avg2.__closure__[0].cell_contents) # cell对象的cell_contents属性保存着自有变量的值

('new_value', 'total')
('series',)
<cell at 0x00000174F91603D8: list object at 0x00000174F917DE08>
[10, 11]


## nonlocal
用于标记自由变量，有时候需要声明变量为自由变量，否在在函数内需要对自由变量赋值会被python当做本地变量。nonlocal的作用和global声明类似，就是为了能对自由变量或全局变量赋值而不会被当做局部变量看待。

In [5]:
# 改进均值函数的闭包形式，只记录个数和总和值即可
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        # 这里如果不做自由变量声明，下面的赋值语句如count = count + 1会将count作为局部变量处理，运行时由于赋值表达式有对count的引用就会报错UnboundLocalError（）。
        nonlocal count, total
        count += 1
        total += new_value
        return total / new_value
    
    return averager

# 常见装饰器行为
最常见的实现方式是在装饰器内部定义一个函数实现对被装饰函数的功能增强,且其返回值和原函数一致，然后返回这个函数来替代原函数或对象。

In [13]:
# 计时器装饰器
import time
import functools

def clock(func):
    # 该含参数装饰器（后面会讲）会讲传入的函数的__name__和__doc__属性复制给被装饰的函数，这样返回的函数拥有和原函数一样的信息属性。
    @functools.wraps(func)
    # 函数对传入的原函数做增强，返回自身替代原函数，这里需注意,可支持全部类型形参。
    def clocked(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - start
        
        name = func.__name__
        # 参入的参数值
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    if n < 2:
        return n
    else:
        return n * factorial(n-1)

snooze(.123)
factorial(6)

[0.13483509s] snooze(0.123) -> None
[0.00000154s] factorial(1) -> 1
[0.00207370s] factorial(2) -> 2
[0.00401940s] factorial(3) -> 6
[0.00594093s] factorial(4) -> 24
[0.00786864s] factorial(5) -> 120
[0.00979789s] factorial(6) -> 720


720

# 标准库中的装饰器
property、classmethod、staticmethod会在后面讨论，这里主要讨论functools中常用的wraps、lru_cache、singledispatch。
## lru_cache
主要用与保存最近的函数执行结果，相当于一个最近使用缓存，当传入的形参之前计算过，则会直接返回对应结果无需再次执行函数。

In [14]:
# fibonacci数列的第n项:1,1,2,3,5,8...。当前元素值为前其两个元素值和。

@clock
def fibonacci(n):
    if n < 2:
        return n
    else:
        # 非常直观的写法，但是由于利用迭代，会带来大量重复计算的问题。
        return fibonacci(n-2) + fibonacci(n-1)
    
fibonacci(6)

[0.00000051s] fibonacci(0) -> 0
[0.00000103s] fibonacci(1) -> 1
[0.00008688s] fibonacci(2) -> 1
[0.00000051s] fibonacci(1) -> 1
[0.00000103s] fibonacci(0) -> 0
[0.00000103s] fibonacci(1) -> 1
[0.00004267s] fibonacci(2) -> 1
[0.00008019s] fibonacci(3) -> 2
[0.00021230s] fibonacci(4) -> 3
[0.00000051s] fibonacci(1) -> 1
[0.00000051s] fibonacci(0) -> 0
[0.00000051s] fibonacci(1) -> 1
[0.00004678s] fibonacci(2) -> 1
[0.00009099s] fibonacci(3) -> 2
[0.00000051s] fibonacci(0) -> 0
[0.00000051s] fibonacci(1) -> 1
[0.00003187s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000103s] fibonacci(0) -> 0
[0.00000051s] fibonacci(1) -> 1
[0.00005552s] fibonacci(2) -> 1
[0.00009921s] fibonacci(3) -> 2
[0.00015987s] fibonacci(4) -> 3
[0.00029764s] fibonacci(5) -> 5
[0.00080604s] fibonacci(6) -> 8


8

In [16]:
# 可使用lru_cache缓存解决该问题
@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    else:
        return fibonacci(n-2) + fibonacci(n-1)
    
fibonacci(6)

[0.00000103s] fibonacci(0) -> 0
[0.00000103s] fibonacci(1) -> 1
[0.00011669s] fibonacci(2) -> 1
[0.00000154s] fibonacci(3) -> 2
[0.00043129s] fibonacci(4) -> 3
[0.00000206s] fibonacci(5) -> 5
[0.00049658s] fibonacci(6) -> 8


8

lru_cache接受两个参数functools.lru_cache(maxsize=128, typed=False)，typed相当于区分形参的类型，如1和1.0将被视为不同的形参而将结果分开缓存 
__lru_cache使用字典存储形参和对应结果，由于依赖于函数的全部形参（多半使用形参值元组，因此需要全部可散列），因此需要函数的所有参数都必须可散列。__
## 叠放装饰器
上面示例中使用了叠加装饰器，相当于fibonacci = functools.lru_cache()(clock(fibonacci))。当使用叠加装饰器是往往需要装饰器中返回的增强函数的返回值和被装饰的函数一致，这样一来增强函数便可完全替代被装饰函数，这样才能使用多层装饰器叠加。 

## singledispatch
有时函数希望根据传入传入参数的类型来做不同的处理，一种在python中常用的做法是利用if else去调用不同的分支函数，但随着分派函数的增加会使得函数体积迅速增加，同时也加剧了其和分派函数的耦合，显得很笨拙，因此利用单分派装饰器就可以解决此问题。

单分派泛函数，有些类似于Java中的函数重载，由于python不支持函数重载因此可利用此装饰器来实现，不过只能依据第一个参数类型的不同来调用不同的函数。
#### 另外由于利用了装饰器，因此不同的分派函数可以写在不同的模块中，这样一来相比于java类函数的重载，其使用更加灵活。
使用单分派装饰器时，形参类型尽量使用基类以支持未来的子类或实现类类型的对象。

In [18]:
from functools import singledispatch
from collections import abc
import numbers
import html

# 将传入的对象转换成html代码
@functools.singledispatch
def himlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@himlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<pre>{0}</pre>'.format(content)

@himlize.register(numbers.Integral)
def _(n):
    return '<pre>{0}(0x{0:x})</pre>'.format(n)

# 实现对大部分Iterable的列表html生成
@himlize.register(tuple)
@himlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)

# 带形参的装饰器
构建带参数的装饰器实际上需要创建装饰器工厂函数，通过对装饰器工厂函数的调用来来创建真正的装饰器函数。这里将之前使用的clock装饰器改为带参数的装饰器。

In [8]:
import time
import functools

def clock(fmt='[{elapsed:0.8f}s] {name}({args}) -> {result}'):
    def decorate(func):
        @functools.wraps(func)
        def clocked(*args, **kwargs):
            _t = time.time()
            _result = func(*args, *kwargs)
            elapsed = time.time() - _t
            name = func.__name__
            args = ', '.join(repr(arg) for arg in args) + ', ' + ', '.join(repr(k)+": "+repr(v) for k, v in kwargs.items())
            result = repr(_result)
            # locals()获取本地变量明和值的映射,可用于关键字参数拆包
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

# 这里必须调用装饰器工场方法返回内部嵌套的装饰器函数。返回的装饰器会立刻运行，相当于snooze = clock()(snooze)。
@clock()
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(0.123)

[0.13305879s] snooze(0.123, ) -> None
[0.12504601s] snooze(0.123, ) -> None
[0.12340951s] snooze(0.123, ) -> None


In [12]:
# snoozed 自由变量绑定情况，fmt无需定义在全局环境内同时在使用参数装饰器的时候指定即可。
snooze.__code__.co_freevars

('fmt', 'func')

#### 创建带参数的装饰器往往需要多一层嵌套来返回真正的装饰器函数，另外由于装饰器多使用嵌套，因此经常以闭包的形式增强原函数的功能（增加了自由变量属性）同时不会对全局环境造成“污染”。
#### 另一方面，当使用叠加装饰器时往往需要装饰器中返回的增强函数的返回值和被装饰的函数一致，这样一来增强函数便可完全替代被装饰函数，这样才能使用多层装饰器叠加。 
装饰器的实现最好通过实现\__call__方法的类实现。